# Домашнее задание №3

# Задание 1
Разработать программу, принимающую на вход изображение из категории Imagenet своего варианта и
выдающее название категории изображенного объекта и вероятность соответствия его этой категории.

In [3]:
# Номер варианта
group = "БИВ206"
number = 4
category = int(group[-1]) * 20 + number * 10
print("Категория:", category)

Категория: 160
